In [37]:
import pandas as pd
data=pd.read_csv("merged_doc.csv")

In [38]:
data.head()

,doc_no,sentence_no,word,tag
0,2803,0,Her,DET
1,2803,0,baş,NOUN
2,2803,0,ağrısı,NOUN
3,2803,0,sinüzit,NOUN
4,2803,0,her,DET


In [39]:
no_start=0
sentence=[]
all_sentence=[]
for no, w, t in zip(data.sentence_no, data.word, data.tag):
    if no_start!=no:
        no_start=no
        all_sentence.append(sentence)
        sentence=[]
        
    sentence.append((w, t))

In [40]:
all_sentence

[[('Her', 'DET'),
  ('baş', 'NOUN'),
  ('ağrısı', 'NOUN'),
  ('sinüzit', 'NOUN'),
  ('her', 'DET'),
  ('burun', 'VERB'),
  ('akıntısı', 'NOUN'),
  ('grip', 'NOUN'),
  ('değildir', 'NOUN'),
  ('.', '.')],
 [('Recep', 'NOUN'),
  ('Tayyip', 'NOUN'),
  ('Erdoğan', 'NOUN'),
  ('Üniversitesi', 'NOUN'),
  ('Tıp', 'NOUN'),
  ('Fakültesi', 'NOUN'),
  ('Kulak', 'NOUN'),
  ('Burun', 'NOUN'),
  ('Boğaz', 'NOUN'),
  ('Hastalıkları', 'NOUN'),
  ('Anabilim', 'NOUN'),
  ('Dalı', 'NOUN'),
  ('Başkanı', 'NOUN'),
  ('Prof', 'NOUN'),
  ('.', '.'),
  ('Dr.', 'NOUN'),
  ('.', '.'),
  ('Engin', 'NOUN'),
  ('Dursun', 'NOUN'),
  ('Karadeniz', 'NOUN'),
  ("Bölgesi'nde", 'NOUN'),
  ('alerjik', 'ADJ'),
  ('burun', 'VERB'),
  ('rahatsızlıklarının', 'NOUN'),
  ('çok', 'ADJ'),
  ('yaygın', 'ADJ'),
  ('olduğunu', 'ADJ'),
  ('belirterek', 'ADV'),
  ('Hastalar', 'NOUN'),
  ('benim', 'PRON'),
  ('uzun', 'ADJ'),
  ('zamandır', 'NOUN'),
  ('burun', 'NOUN'),
  ('akıntım', 'NOUN'),
  ('var.', 'VERB'),
  ('.', '.')],
 [('Çok

In [41]:
tagged_sentence=all_sentence

In [42]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(tagged_sentence,test_size=0.2,random_state=1234)
print("Number of Sentences in Training Data ",len(train_set))
print("Number of Sentences in Testing Data ",len(test_set))

Number of Sentences in Training Data  179
Number of Sentences in Testing Data  45


In [43]:
import re
def features(sentence,index):
    ### sentence is of the form [w1,w2,w3,..], index is the position of the word in the sentence
    return {
        'is_first_capital':int(sentence[index][0].isupper()),
        'is_first_word': int(index==0),
        'is_last_word':int(index==len(sentence)-1),
        'is_complete_capital': int(sentence[index].upper()==sentence[index]),
        'prev_word':'' if index==0 else sentence[index-1],
        'next_word':'' if index==len(sentence)-1 else sentence[index+1],
        'is_numeric':int(sentence[index].isdigit()),
        'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index])))),
        'prefix_1':sentence[index][0],
        'prefix_2': sentence[index][:2],
        'prefix_3':sentence[index][:3],
        'prefix_4':sentence[index][:4],
        'suffix_1':sentence[index][-1],
        'suffix_2':sentence[index][-2:],
        'suffix_3':sentence[index][-3:],
        'suffix_4':sentence[index][-4:],
        'word_has_hyphen': 1 if '-' in sentence[index] else 0  
         }
def untag(sentence):
    return [word for word,tag in sentence]


def prepareData(tagged_sentences):
    X,y=[],[]
    for sentences in tagged_sentences:
        X.append([features(untag(sentences), index) for index in range(len(sentences))])
        y.append([tag for word,tag in sentences])
    return X,y
X_train,y_train=prepareData(train_set)
X_test,y_test=prepareData(test_set)

In [52]:
from sklearn_crfsuite import CRF
crf = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [53]:
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
y_pred=crf.predict(X_test)
print("F1 score on Test Data ")
print(metrics.flat_f1_score(y_test, y_pred,average='weighted',labels=crf.classes_))
print("F score on Training Data ")
y_pred_train=crf.predict(X_train)
metrics.flat_f1_score(y_train, y_pred_train,average='weighted',labels=crf.classes_)

### Look at class wise score
print(metrics.flat_classification_report(
    y_test, y_pred, labels=crf.classes_, digits=3
))

F1 score on Test Data 
0.8166325013709103
F score on Training Data 
              precision    recall  f1-score   support

         ADV      0.824     0.341     0.483        41
        PRON      1.000     0.200     0.333        10
        NOUN      0.826     0.959     0.888       466
         ADP      0.654     0.810     0.723        21
        VERB      0.920     0.730     0.814        63
           .      1.000     1.000     1.000        26
         ADJ      0.744     0.571     0.646       112
        CONJ      0.963     0.839     0.897        31
         NUM      0.875     0.538     0.667        13
         DET      0.862     0.926     0.893        27
         PRT      0.000     0.000     0.000         1

   micro avg      0.830     0.831     0.831       811
   macro avg      0.788     0.629     0.668       811
weighted avg      0.831     0.831     0.817       811

